In [43]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os, os.path as osp
pd.set_option('mode.chained_assignment',None)

# Analyze other glaucoma-related features

* Appearance neuroretinal rim superiorly (ANRS)
* Appearance neuroretinal rim inferiorly (ANRI)
* Retinal nerve fiber layer defect superiorly (RNFLDS)
* Retinal nerve fiber layer defect inferiorly (RNFLDI)
* Baring circumlinear vessel superiorly (BCLVS)
* Baring circumlinear vessel inferiorly (BCLVI)
* Nasalisation of vessel trunk (NVT)
* Disc hemorrhages (DH)
* Laminar dots (LD)
* Large cup (LC)

Thing is, it might happen that G1 and G2 disagree at the features level, and in that case G3 will not say anything. G3 will only say something if G1 and G2 disagree at the referral level.

In [2]:
df = pd.read_csv('data/JustRAIGS_Train_labels.csv', sep=';', index_col=False)
df.columns

Index(['Eye ID', 'Final Label', 'Fellow Eye ID', 'Age', 'Label G1', 'Label G2',
       'Label G3', 'G1 ANRS', 'G1 ANRI', 'G1 RNFLDS', 'G1 RNFLDI', 'G1 BCLVS',
       'G1 BCLVI', 'G1 NVT', 'G1 DH', 'G1 LD', 'G1 LC', 'G2 ANRS', 'G2 ANRI',
       'G2 RNFLDS', 'G2 RNFLDI', 'G2 BCLVS', 'G2 BCLVI', 'G2 NVT', 'G2 DH',
       'G2 LD', 'G2 LC', 'G3 ANRS', 'G3 ANRI', 'G3 RNFLDS', 'G3 RNFLDI',
       'G3 BCLVS', 'G3 BCLVI', 'G3 NVT', 'G3 DH', 'G3 LD', 'G3 LC'],
      dtype='object')

In [3]:
df.shape

(101423, 37)

In [4]:
def labels(label):
    if label=='NRG': return 0
    elif label=='RG': return 1
    elif label=='U': return 2
    elif label==np.nan: return 3
df['Final Label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array(['NRG', 'RG'], dtype=object),
 array(['NRG', 'RG', 'U', nan], dtype=object),
 array(['NRG', nan, 'RG', 'U'], dtype=object))

In [5]:
df['Final Label'] = df['Final Label'].map(labels)
df['Label G1'] = df['Label G1'].map(labels)
df['Label G2'] = df['Label G2'].map(labels)
df['Label G3'] = df['Label G3'].map(labels)

df['Final Label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array([0, 1]), array([ 0.,  1.,  2., nan]), array([ 0., nan,  1.,  2.]))

In [6]:
df_features = df[df['Final Label']==1]
df_features.shape

(3270, 37)

In [7]:
df_features = df[df['Label G3']==1]
df_features.shape

(992, 37)

In [8]:
df_features = df[(df['Label G1']==1) & (df['Label G2']==1)]
df_features.shape

(2279, 37)

In [9]:
2279+992

3271

So final label = 1 comes from images where G1 agreed with G2 PLUS a part of the images where G1!=G2, where G3 made final decision. But there are many more images graded as 1 by G1 and G2 that did not make it into this set.

Point is, G1 said 1 5000 times, and only 2300 where agreements with G2! Should we use those for training?

In [4]:
0.139+(1-0.831)

0.30800000000000005

Ok, here is what we do. 
* There are images where only G3 gave an opinion. Those are hard-labels and there is no alternative.
* Then there are images where G1!=G2, and G3 gave an opinion. In this case, maybe G1==G3, but not at the feature level. We then assign a 0.1/0.9 to disagreement, in favor of G3. Same with G2.
* This still leaves the case where G1==G2==1, but not at the feature level! In those cases, we will add a 0.5 if they disagree at some feature.
* Then, we collect all images where G1 == 1 or G2 ==1 but Final Label == 0. In this case, G3 did not give info at feature level as they thought there was no glaucoma. We will keep those as "soft labeled", and add a 0.25 on feature values.

Goodness.

### Only G3 gave opinion

In [10]:
df_G3_alone = df[(df['Label G3']==1) & (df['Label G1']!=1) & (df['Label G2']!=1)]
df_G3_alone.shape

(173, 37)

In [11]:
df_G3_alone=df_G3_alone.drop([n for n in df.columns if 'G1' in n], axis=1)
df_G3_alone=df_G3_alone.drop([n for n in df.columns if 'G2' in n], axis=1)
df_G3_alone=df_G3_alone.drop(['Label G3', 'Final Label', 'Fellow Eye ID', 'Age'], axis=1)
df_G3_alone=df_G3_alone.rename(columns=lambda x: x.replace('G3 ','') if 'G3' in x else x)
df_G3_alone.columns

Index(['Eye ID', 'ANRS', 'ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT',
       'DH', 'LD', 'LC'],
      dtype='object')

In [12]:
df_G3_alone = df_G3_alone[['Eye ID', 'ANRS','ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS','BCLVI', 'NVT', 'DH', 'LD', 'LC']]

### G1 thought Glaucoma, G3 agreed, G2 disagreed

In [13]:
df_G1G3 = df[(df['Label G3']==1) & (df['Label G1']==1) & (df['Label G2']!=1)]
df_G1G3.shape

(436, 37)

In [14]:
def build_labels_g1g3(df, col):
    if df['G1 '+col]==df['G3 '+col]: return df['G3 '+col]
    else:
        if df['G3 '+col]==0: return 0.1
        elif df['G3 '+col]==1: return 0.9
        else: print('wtf')

In [15]:
df_G1G3['ANRS']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'ANRS'), axis=1)
df_G1G3['ANRI']= df_G1G3.apply(lambda x: build_labels_g1g3(x, 'ANRI'), axis=1)

df_G1G3['RNFLDS']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'RNFLDS'), axis=1)
df_G1G3['RNFLDI']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'RNFLDI'), axis=1)

df_G1G3['BCLVS']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'BCLVS'), axis=1)
df_G1G3['BCLVI']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'BCLVI'), axis=1)

df_G1G3['NVT']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'NVT'), axis=1)
df_G1G3['DH']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'DH'), axis=1)

df_G1G3['LD']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'LD'), axis=1)
df_G1G3['LC']=df_G1G3.apply(lambda x: build_labels_g1g3(x, 'LC'), axis=1)

In [16]:
df_G1G3 = df_G1G3[['Eye ID', 'ANRS','ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT', 'DH', 'LD', 'LC']]

### G2 thought Glaucoma, G3 agreed, G1 disagreed

In [17]:
df_G2G3 = df[(df['Label G3']==1) & (df['Label G2']==1) & (df['Label G1']!=1)]
df_G2G3.shape

(382, 37)

In [18]:
def build_labels_g2g3(df, col):
    if df['G2 '+col]==df['G3 '+col]: return df['G3 '+col]
    else:
        if df['G3 '+col]==0: return 0.1
        elif df['G3 '+col]==1: return 0.9
        else: print('wtf')

In [19]:
df_G2G3['ANRS']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'ANRS'), axis=1)
df_G2G3['ANRI']= df_G2G3.apply(lambda x: build_labels_g2g3(x, 'ANRI'), axis=1)

df_G2G3['RNFLDS']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'RNFLDS'), axis=1)
df_G2G3['RNFLDI']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'RNFLDI'), axis=1)

df_G2G3['BCLVS']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'BCLVS'), axis=1)
df_G2G3['BCLVI']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'BCLVI'), axis=1)

df_G2G3['NVT']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'NVT'), axis=1)
df_G2G3['DH']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'DH'), axis=1)

df_G2G3['LD']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'LD'), axis=1)
df_G2G3['LC']=df_G2G3.apply(lambda x: build_labels_g2g3(x, 'LC'), axis=1)

In [20]:
df_G2G3 = df_G2G3[['Eye ID', 'ANRS','ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT', 'DH', 'LD', 'LC']]

### G1 thought Glaucoma, G2 agreed, but maybe not at feature level

In [21]:
df_G1G2 = df[(df['Label G1']==1) & (df['Label G2']==1)]
df_G1G2.shape

(2279, 37)

In [22]:
df_G1G2[df_G1G2['G1 ANRS']==df_G1G2['G2 ANRS']].shape

(1791, 37)

In [23]:
def build_labels_g1g2(df, col):
    if df['G1 '+col]==df['G2 '+col]: return df['G1 '+col]
    else: return 0.5

In [24]:
df_G1G2['ANRS']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'ANRS'), axis=1)
df_G1G2['ANRI']= df_G1G2.apply(lambda x: build_labels_g1g2(x, 'ANRI'), axis=1)

df_G1G2['RNFLDS']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'RNFLDS'), axis=1)
df_G1G2['RNFLDI']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'RNFLDI'), axis=1)

df_G1G2['BCLVS']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'BCLVS'), axis=1)
df_G1G2['BCLVI']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'BCLVI'), axis=1)

df_G1G2['NVT']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'NVT'), axis=1)
df_G1G2['DH']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'DH'), axis=1)

df_G1G2['LD']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'LD'), axis=1)
df_G1G2['LC']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'LC'), axis=1)

In [25]:
np.unique(df_G1G2['ANRS'], return_counts=True)

(array([0. , 0.5, 1. ]), array([ 407,  488, 1384]))

In [26]:
np.unique(df_G1G2['ANRI'], return_counts=True)

(array([0. , 0.5, 1. ]), array([ 258,  477, 1544]))

In [27]:
df_G1G2 = df_G1G2[['Eye ID', 'ANRS','ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT', 'DH', 'LD', 'LC']]

### G1 thought Glaucoma, G2 and G3 disagreed

In [28]:
df_G1 = df[(df['Label G1']==1) & (df['Label G2']==0) & (df['Label G3']==0)]
df_G1.shape

(2166, 37)

In [29]:
def build_labels_g1(df, col):
    if df['G1 '+col]==1: return 0.25
    else: return 0

In [30]:
df_G1['ANRS']=df_G1.apply(lambda x: build_labels_g1(x, 'ANRS'), axis=1)
df_G1['ANRI']= df_G1.apply(lambda x: build_labels_g1(x, 'ANRI'), axis=1)

df_G1['RNFLDS']=df_G1.apply(lambda x: build_labels_g1(x, 'RNFLDS'), axis=1)
df_G1['RNFLDI']=df_G1.apply(lambda x: build_labels_g1(x, 'RNFLDI'), axis=1)

df_G1['BCLVS']=df_G1.apply(lambda x: build_labels_g1(x, 'BCLVS'), axis=1)
df_G1['BCLVI']=df_G1.apply(lambda x: build_labels_g1(x, 'BCLVI'), axis=1)

df_G1['NVT']=df_G1.apply(lambda x: build_labels_g1(x, 'NVT'), axis=1)
df_G1['DH']=df_G1.apply(lambda x: build_labels_g1(x, 'DH'), axis=1)

df_G1['LD']=df_G1.apply(lambda x: build_labels_g1(x, 'LD'), axis=1)
df_G1['LC']=df_G1.apply(lambda x: build_labels_g1(x, 'LC'), axis=1)

In [31]:
df_G1 = df_G1[['Eye ID', 'ANRS','ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT', 'DH', 'LD', 'LC']]

### G2 thought Glaucoma, G1 and G3 disagreed

In [32]:
df_G2 = df[(df['Label G2']==1) & (df['Label G1']==0) & (df['Label G3']==0)]
df_G2.shape

(1054, 37)

In [33]:
def build_labels_g2(df, col):
    if df['G2 '+col]==1: return 0.25
    else: return 0

In [34]:
df_G2['ANRS']=df_G2.apply(lambda x: build_labels_g2(x, 'ANRS'), axis=1)
df_G2['ANRI']= df_G2.apply(lambda x: build_labels_g2(x, 'ANRI'), axis=1)

df_G2['RNFLDS']=df_G2.apply(lambda x: build_labels_g2(x, 'RNFLDS'), axis=1)
df_G2['RNFLDI']=df_G2.apply(lambda x: build_labels_g2(x, 'RNFLDI'), axis=1)

df_G2['BCLVS']=df_G2.apply(lambda x: build_labels_g2(x, 'BCLVS'), axis=1)
df_G2['BCLVI']=df_G2.apply(lambda x: build_labels_g2(x, 'BCLVI'), axis=1)

df_G2['NVT']=df_G2.apply(lambda x: build_labels_g2(x, 'NVT'), axis=1)
df_G2['DH']=df_G2.apply(lambda x: build_labels_g2(x, 'DH'), axis=1)

df_G2['LD']=df_G2.apply(lambda x: build_labels_g2(x, 'LD'), axis=1)
df_G2['LC']=df_G2.apply(lambda x: build_labels_g2(x, 'LC'), axis=1)

In [35]:
df_G2 = df_G2[['Eye ID', 'ANRS','ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT', 'DH', 'LD', 'LC']]

## Done
So we end up with a bunch of dataframes, let us see:

In [36]:
print('trustworthy:', df_G3_alone.shape, df_G1G3.shape, df_G2G3.shape, df_G1G2.shape)
print('unreliable:', df_G1.shape, df_G2.shape)

trustworthy: (173, 11) (436, 11) (382, 11) (2279, 11)
unreliable: (2166, 11) (1054, 11)


In [37]:
df_features = pd.concat([df_G3_alone, df_G1G3, df_G2G3, df_G1G2, df_G1, df_G2], axis=0)
df_features.shape

(6490, 11)

In [38]:
df_features.isnull().any()

Eye ID    False
ANRS      False
ANRI      False
RNFLDS    False
RNFLDI    False
BCLVS     False
BCLVI     False
NVT       False
DH        False
LD        False
LC        False
dtype: bool

But we will not be using df_G1 and df_G2 for validation, so do the train/val split first, then add those to train.

### First option: use only reliable annotations (although soft):

In [72]:
df_features_mad = pd.concat([df_G3_alone, df_G1G3, df_G2G3, df_G1G2], axis=0)
df_features_madder = pd.concat([df_G1, df_G2], axis=0)
df_features_mad.shape, df_features_madder.shape

((4089, 11), (3220, 11))

In [75]:
df_features_mad = df_features_mad.rename(columns={'Eye ID': 'image_id'})
df_features_mad.image_id = [osp.join('data/images', n + '.JPG') for n in df_features_mad.image_id.values]

In [76]:
df_features_madder = df_features_madder.rename(columns={'Eye ID': 'image_id'})
df_features_madder.image_id = [osp.join('data/images', n + '.JPG') for n in df_features_madder.image_id.values]

In [81]:
df_features_mad.shape, np.unique(df_features_mad[[n for n in df_features_mad.columns if n!='image_id']])

((4089, 11), array([0. , 0.1, 0.9, 1. ]))

In [80]:
df_features_madder.shape, np.unique(df_features_madder[[n for n in df_features_madder.columns if n!='image_id']])

((3220, 11), array([0.  , 0.25]))

In [48]:
num_ims = len(df_features_good)
meh, df_val1 = train_test_split(df_features_good, test_size=num_ims//5, random_state=0)
meh, df_val2 = train_test_split(meh,    test_size=num_ims//5, random_state=0)
meh, df_val3 = train_test_split(meh,    test_size=num_ims//5, random_state=0)
df_val5, df_val4 = train_test_split(meh,test_size=num_ims//5, random_state=0)

df_train1 = pd.concat([df_val2,df_val3,df_val4,df_val5], axis=0)
df_train2 = pd.concat([df_val1,df_val3,df_val4,df_val5], axis=0)
df_train3 = pd.concat([df_val1,df_val2,df_val4,df_val5], axis=0)
df_train4 = pd.concat([df_val1,df_val2,df_val3,df_val5], axis=0)
df_train5 = pd.concat([df_val1,df_val2,df_val3,df_val4], axis=0)

df_train1.to_csv('data/tr_features_soft_mad_f1.csv', index=None)
df_val1.to_csv('data/vl_features_soft_mad_f1.csv', index=None)

df_train2.to_csv('data/tr_features_soft_mad_f2.csv', index=None)
df_val2.to_csv('data/vl_features_soft_mad_f2.csv', index=None)

df_train3.to_csv('data/tr_features_soft_mad_f3.csv', index=None)
df_val3.to_csv('data/vl_features_soft_mad_f3.csv', index=None)

df_train4.to_csv('data/tr_features_soft_mad_f4.csv', index=None)
df_val4.to_csv('data/vl_features_soft_mad_f4.csv', index=None)

df_train5.to_csv('data/tr_features_soft_mad_f5.csv', index=None)
df_val5.to_csv('data/vl_features_soft_mad_f5.csv', index=None)

### Second option: add unreliable annotations to training sets:

In [49]:
print(df_train1.shape)
df_train1 = pd.concat([df_train1, df_features_goodish], axis=0)
print(df_train1.shape)

(2616, 11)
(5836, 11)


In [50]:
df_train2 = pd.concat([df_train2, df_features_goodish], axis=0)
df_train3 = pd.concat([df_train3, df_features_goodish], axis=0)
df_train4 = pd.concat([df_train4, df_features_goodish], axis=0)
df_train5 = pd.concat([df_train5, df_features_goodish], axis=0)

In [51]:
df_train1.to_csv('data/tr_features_soft_madder_f1.csv', index=None)
df_val1.to_csv('data/vl_features_soft_madder_f1.csv', index=None)

df_train2.to_csv('data/tr_features_soft_madder_f2.csv', index=None)
df_val2.to_csv('data/vl_features_soft_madder_f2.csv', index=None)

df_train3.to_csv('data/tr_features_soft_madder_f3.csv', index=None)
df_val3.to_csv('data/vl_features_soft_madder_f3.csv', index=None)

df_train4.to_csv('data/tr_features_soft_madder_f4.csv', index=None)
df_val4.to_csv('data/vl_features_soft_madder_f4.csv', index=None)

df_train5.to_csv('data/tr_features_soft_madder_f5.csv', index=None)
df_val5.to_csv('data/vl_features_soft_madder_f5.csv', index=None)

# Baseline Common-Sense alternative to all that mess
All are hard labels. Just keep final label, if there is G3 we keep that, otherwise, we do random selection.

In [57]:
df = pd.read_csv('data/JustRAIGS_Train_labels.csv', sep=';', index_col=False)
df.columns

Index(['Eye ID', 'Final Label', 'Fellow Eye ID', 'Age', 'Label G1', 'Label G2',
       'Label G3', 'G1 ANRS', 'G1 ANRI', 'G1 RNFLDS', 'G1 RNFLDI', 'G1 BCLVS',
       'G1 BCLVI', 'G1 NVT', 'G1 DH', 'G1 LD', 'G1 LC', 'G2 ANRS', 'G2 ANRI',
       'G2 RNFLDS', 'G2 RNFLDI', 'G2 BCLVS', 'G2 BCLVI', 'G2 NVT', 'G2 DH',
       'G2 LD', 'G2 LC', 'G3 ANRS', 'G3 ANRI', 'G3 RNFLDS', 'G3 RNFLDI',
       'G3 BCLVS', 'G3 BCLVI', 'G3 NVT', 'G3 DH', 'G3 LD', 'G3 LC'],
      dtype='object')

In [59]:
def labels(label):
    if label=='NRG': return 0
    elif label=='RG': return 1
    elif label=='U': return 2
    elif label==np.nan: return 3
df['Final Label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array(['NRG', 'RG'], dtype=object),
 array(['NRG', 'RG', 'U', nan], dtype=object),
 array(['NRG', nan, 'RG', 'U'], dtype=object))

In [60]:
df['Final Label'] = df['Final Label'].map(labels)
df['Label G1'] = df['Label G1'].map(labels)
df['Label G2'] = df['Label G2'].map(labels)
df['Label G3'] = df['Label G3'].map(labels)

df['Final Label'].unique(), df['Label G1'].unique(), df['Label G2'].unique()

(array([0, 1]), array([ 0.,  1.,  2., nan]), array([ 0., nan,  1.,  2.]))

In [61]:
df[df['Final Label']==1].shape

(3270, 37)

In [62]:
df_G3_alone = df[(df['Label G3']==1)]
df_G3_alone.shape

(992, 37)

In [63]:
df_G3_alone=df_G3_alone.drop([n for n in df.columns if 'G1' in n], axis=1)
df_G3_alone=df_G3_alone.drop([n for n in df.columns if 'G2' in n], axis=1)
df_G3_alone=df_G3_alone.drop(['Label G3', 'Final Label', 'Fellow Eye ID', 'Age'], axis=1)
df_G3_alone=df_G3_alone.rename(columns=lambda x: x.replace('G3 ','') if 'G3' in x else x)
df_G3_alone.columns

Index(['Eye ID', 'ANRS', 'ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT',
       'DH', 'LD', 'LC'],
      dtype='object')

### G1 thought Glaucoma, G2 agreed, but not at feature level - pick G2 who tends to agree more with G3

In [64]:
df_G1G2 = df[(df['Label G1']==1) & (df['Label G2']==1)]
df_G1G2.shape

(2279, 37)

In [65]:
def build_labels_g1g2(df, col):
    return df['G2 '+col]

In [66]:
df_G1G2['ANRS']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'ANRS'), axis=1)
df_G1G2['ANRI']= df_G1G2.apply(lambda x: build_labels_g1g2(x, 'ANRI'), axis=1)

df_G1G2['RNFLDS']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'RNFLDS'), axis=1)
df_G1G2['RNFLDI']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'RNFLDI'), axis=1)

df_G1G2['BCLVS']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'BCLVS'), axis=1)
df_G1G2['BCLVI']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'BCLVI'), axis=1)

df_G1G2['NVT']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'NVT'), axis=1)
df_G1G2['DH']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'DH'), axis=1)

df_G1G2['LD']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'LD'), axis=1)
df_G1G2['LC']=df_G1G2.apply(lambda x: build_labels_g1g2(x, 'LC'), axis=1)

In [67]:
df_G1G2=df_G1G2.drop([n for n in df.columns if 'G1' in n], axis=1)
df_G1G2=df_G1G2.drop([n for n in df.columns if 'G2' in n], axis=1)
df_G1G2=df_G1G2.drop([n for n in df.columns if 'G3' in n], axis=1)
df_G1G2=df_G1G2.drop(['Final Label', 'Fellow Eye ID', 'Age'], axis=1)
# df_G1G2=df_G1G2.rename(columns=lambda x: x.replace('G3 ','') if 'G3' in x else x)
df_G1G2.columns

Index(['Eye ID', 'ANRS', 'ANRI', 'RNFLDS', 'RNFLDI', 'BCLVS', 'BCLVI', 'NVT',
       'DH', 'LD', 'LC'],
      dtype='object')

In [68]:
df_features_sane = pd.concat([df_G3_alone, df_G1G2], axis=0)
df_features_sane.shape

(3271, 11)

In [69]:
df_features_sane = df_features_sane.rename(columns={'Eye ID': 'image_id'})
df_features_sane.image_id = [osp.join('data/images', n + '.JPG') for n in df_features_sane.image_id.values]

### build csvs

In [70]:
num_ims = len(df_features_sane)
meh, df_val1 = train_test_split(df_features_sane, test_size=num_ims // 5, random_state=0)
meh, df_val2 = train_test_split(meh, test_size=num_ims // 5, random_state=0)
meh, df_val3 = train_test_split(meh, test_size=num_ims // 5, random_state=0)
df_val5, df_val4 = train_test_split(meh, test_size=num_ims // 5, random_state=0)

df_train1 = pd.concat([df_val2,df_val3,df_val4,df_val5], axis=0)
df_train2 = pd.concat([df_val1,df_val3,df_val4,df_val5], axis=0)
df_train3 = pd.concat([df_val1,df_val2,df_val4,df_val5], axis=0)
df_train4 = pd.concat([df_val1,df_val2,df_val3,df_val5], axis=0)
df_train5 = pd.concat([df_val1,df_val2,df_val3,df_val4], axis=0)

df_train1.to_csv('data/tr_features_sane_f1.csv', index=None)
df_val1.to_csv('data/vl_features_sane_f1.csv', index=None)

df_train2.to_csv('data/tr_features_sane_f2.csv', index=None)
df_val2.to_csv('data/vl_features_sane_f2.csv', index=None)

df_train3.to_csv('data/tr_features_sane_f3.csv', index=None)
df_val3.to_csv('data/vl_features_sane_f3.csv', index=None)

df_train4.to_csv('data/tr_features_sane_f4.csv', index=None)
df_val4.to_csv('data/vl_features_sane_f4.csv', index=None)

df_train5.to_csv('data/tr_features_sane_f5.csv', index=None)
df_val5.to_csv('data/vl_features_sane_f5.csv', index=None)

In [71]:
np.unique(df_features_sane[[n for n in df_features_sane.columns if n!='image_id']])

array([0., 1.])

# Handling Validation
Thing is, it is okay to train with soft labels, but then again when validating, we will need hard labels. The organizers do as follows:
* If there is G3, that rules
* If there is not G3, then if G1==G2 that works, otherwise, ignore that feature.

In [ ]:
def hamming_loss(true_labels, predicted_labels):
    """Calculate the Hamming loss for the given true and predicted labels."""
    # Convert to numpy arrays for efficient computation
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)

    # Calculate the hamming distance that is basically the total number of mismatches
    Hamming_distance = np.sum(np.not_equal(true_labels, predicted_labels))
    
    # Calculate the total number of labels
    total_corrected_labels= true_labels.size

    # Compute the Modified Hamming loss
    loss = Hamming_distance / total_corrected_labels
    return loss

In [ ]:
true_labels = [0,0,0,0,0,1,1,1,1,1]
pred_labels = [0,0,0,0,0,1,1,1,1,1]
hamming_loss(true_labels, pred_labels)

In [ ]:
true_labels = [0,0,0,0,0,1,1,1,1,1]
pred_labels = [0,0,0,0,1,0,1,1,1,1]
hamming_loss(true_labels, pred_labels)

In [ ]:
true_labels = [0,0,0,0,1,1,1,1] # remove two labels that might have been "polemic"
pred_labels = [0,0,0,1,0,1,1,1]
hamming_loss(true_labels, pred_labels)

So in validation we need to do as follows:
* If labels are 0 or 1, then keep them
* if they are 0.1 or 0.9 that means they come from G3, also keep them
* If they are 0.25 or 0.5, ignore them, as they do not come from G3 or G1==G2

In [ ]:
def hamming_loss(true_labels, predicted_labels):
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)
    # keep only features for which there was agreement, see data_splitting_features.ipynb
    y_true=true_labels[(true_labels<=0.1) | (true_labels>=0.9)]
    y_true = y_true>0.5 # need to binarize now
    y_pred=pred_labels[(true_labels<=0.1) | (true_labels>=0.9)]
    # Calculate the hamming distance that is basically the total number of mismatches
    hamming_distance = np.sum(np.not_equal(y_true, y_pred))
    
    # Calculate the total number of labels
    total_corrected_labels= y_true.size

    # Compute the Modified Hamming loss
    loss = hamming_distance / total_corrected_labels
    return loss

In [ ]:
true_labels = np.array([0,0.1,0.25,0.75,0.9,1])
pred_labels = np.array([0,  0,   0,   1,  1, 1])

In [ ]:
true_labels[(true_labels<=0.1) | (true_labels>=0.9)],\
pred_labels[(true_labels<=0.1) | (true_labels>=0.9)]

In [ ]:
hamming_loss(true_labels, pred_labels)